Como punto de partida para la creación del dataset inicial, generaremos un script de webscraping que recoja las ICOs listadas en www.icoholder.com.

El script constará de dos partes: en la primera se consigue el listado de todas las ICOs que hay en la página y en la segunda se iterará sobre

In [ ]:
#Importamos las librerias de selenium para poder trabajar con ellas durante la actividad
# creamos la variable webdriver de firebox, con la que vamos a recuperar los distintos elementos de la web
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

import pandas as pd
import numpy as np
import time
import os
os.environ["PATH"] = '$PATH:.'
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox()

In [ ]:
driver.get("https://icoholder.com/en/icos/past")
driver.implicitly_wait(10)

In [ ]:
# Obtengo el numero de la ultima pagina disponible.
a = driver.find_element(By.CSS_SELECTOR, ".pagination > li:nth-child(10)").text
# Genero una lista con los numeros de las paginas de manera que se pueda ir recorriendo cada una y obteniendo la lista de URL de cada ICO listada en esa pagina.
#total_pages = np.arange(int(a)-1,1,-1)
total_pages = np.arange(425,1,-1)
URL_list_icos =[]

for i in total_pages:
    # Me posiciono en la pagina
    driver.get("https://icoholder.com/en/icos/past?page="+str(i))
    driver.implicitly_wait(10)

    #Por cada pagina hay 31 ICOs, por lo que recorro cada uno de ellos y obtengo la URL de cada uno de ellos.
    for i in range(1,32,2):
        #Me quedo con el elemento del ico que tiene nombre y url
        selector = f"div.ico-list-row:nth-child({i}) > div:nth-child(3) > h3:nth-child(1) > a:nth-child(1)"
        aItem = driver.find_element(By.CSS_SELECTOR, selector)

        #Guardo el ico en una lista [Nombre, URL]
        URL_list_icos.append([aItem.text,aItem.get_attribute('href')])

    #URL_list_icos.append(get_icos_url(driver))
    time.sleep(1.24)

len(URL_list_icos)


6784

In [ ]:
len(URL_list_icos)

3513

In [ ]:
# Con la lista de URL ICOs creo un dataFrafe y lo guardo en un csv.
df_url_2 = pd.DataFrame(URL_list_icos, columns=['ICO_Name', 'URL'])
df_url_2.to_csv('url_icos_425_v2.csv', index=False)

In [ ]:
df_final = pd.concat([df_url ,df_url_2],axis=0).reset_index()
df_final

,index,ICO_Name,URL
0,0,World Cup Apes,https://icoholder.com/en/world-cup-apes-1024462
1,1,Premium Trade,https://icoholder.com/en/premium-trade
2,2,Moneypot X (MPX) ICO,https://icoholder.com/en/moneypot-x-mpx-ico-3246
3,3,LoopX,https://icoholder.com/en/loopx-17448
4,4,Cromacoin,https://icoholder.com/en/cromacoin-25611
...,...,...,...
10292,6779,BCharity - International Charity Exchange,https://icoholder.com/en/bcharity-internationa...
10293,6780,TheFund.io,https://icoholder.com/en/thefund-io-26513
10294,6781,Stan World,https://icoholder.com/en/stan-world-30970
10295,6782,P2PS,https://icoholder.com/en/p2ps-16779


In [ ]:
df_final[df_final.duplicated(subset='URL')]

,index,ICO_Name,URL
17,17,Premium Trade,https://icoholder.com/en/premium-trade
19,19,CryptoKami,https://icoholder.com/en/cryptokami-18029
20,20,Cromacoin,https://icoholder.com/en/cromacoin-25611
22,22,Maximine ICO,https://icoholder.com/en/maximine-ico-21047
25,25,SupportChain,https://icoholder.com/en/supportchain-23096
...,...,...,...
10217,6704,World Cup Apes,https://icoholder.com/en/world-cup-apes-1024462
10233,6720,Cryptostone,https://icoholder.com/en/cryptostone-1025141
10249,6736,IguVerse™,https://icoholder.com/en/iguverse-1025023
10265,6752,World Cup Apes,https://icoholder.com/en/world-cup-apes-1024462


In [ ]:
df_final.shape

(10297, 3)

In [ ]:
df_final.drop_duplicates(subset='URL', keep='first', inplace=True)

In [ ]:
df_final.drop(columns='index',inplace=True)
df_final.shape

(7770, 2)

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
import re
import pandas as pd
#import matplotlib.pyplot as plt
import urllib
from lxml import html
from lxml import cssselect
import time
from urllib.request import urlopen
from IPython.display import display, Markdown
import os


os.environ["PATH"] = '$PATH:.'
options = webdriver.ChromeOptions()
driver = webdriver.Chrome()

In [ ]:
icos_df = df_final.copy()
icos_df
icos_df["ICO_price"] = ""
icos_df["ico_launch_month"] = ""
icos_df["ico_end_month"] = ""
icos_df["ticker"] = ""
icos_df["accepted_currency"] = ""
icos_df["platform"] = ""
icos_df["category"] = ""
icos_df["country"] = ""

In [ ]:
dicti = {"Ticker": 4,
         "Accepted Currencies": 5,
         "Platform": 6,
         "Categories": 7,
         "Registered Country": 8}

In [ ]:
for index, row in icos_df.iterrows():
    try:
        time.sleep(1.2)
        driver.get(row["URL"])
        req = urllib.request.Request(row["URL"], headers={'User-Agent' : "Magic Browser"})
        url = urlopen(req)
        tree = html.document_fromstring(url.read())
        quotes = tree.cssselect("div.ico-more-info__row")

        try:
            icos_df.iloc[index,2] = driver.find_element(By.CSS_SELECTOR,"div.d-inline-block:nth-child(3)").text
            print(driver.find_element(By.CSS_SELECTOR,"div.d-inline-block:nth-child(3)").text)
        except Exception as e:
            row["ICO_price"] = "Not Found"


        try:
            icos_df.iloc[index,3] = driver.find_element(By.CSS_SELECTOR,".ico-list-date-from").text
            print(driver.find_element(By.CSS_SELECTOR,".ico-list-date-from").text)
        except Exception as e:
            row["ico_launch_month"] = "Not Found"

        try:
            icos_df.iloc[index,4] = driver.find_element(By.XPATH,"/html/body/header/div/div/div[4]/div/div[2]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div[2]/div/div[2]").text
            print(driver.find_element(By.XPATH,"/html/body/header/div/div/div[4]/div/div[2]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div[2]/div/div[2]").text)
        except Exception as e:
            row.at["ico_end_month"] = "Not Found"

        for i in range(len(quotes)):
            text_strings = quotes[i-1].text_content().split("\n")
            print(text_strings)
            print(str(dicti.get(text_strings[1])))
            print(icos_df.iloc[index,dicti.get(text_strings[1])])
            print(text_strings[3])
            icos_df.iloc[index, dicti.get(text_strings[1])]= text_strings[3]
            print("________")
    except:
        print("Error, jumping to the next one")

In [ ]:
icos_df.to_csv("Icos_scrapped.csv")

In [ ]:
driver.quit()